In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

plt.style.use("seaborn-v0_8")

# ------------------------------------------------------------
# FUNCTION: Get company name from Yahoo Finance
# ------------------------------------------------------------
def get_company_name(ticker):
    try:
        info = yf.Ticker(ticker).info
        return info.get("longName") or info.get("shortName") or ticker
    except:
        return ticker

# ------------------------------------------------------------
# USER INPUT
# ------------------------------------------------------------
amount_of_money = int(input("What is the total investment amount: $"))

tickers_df = pd.read_csv("tickers.csv")
tickers_df["ticker"] = tickers_df["ticker"].astype(str).str.strip()
tickers = tickers_df["ticker"].dropna().tolist()

# ------------------------------------------------------------
# DOWNLOAD PRICE DATA
# ------------------------------------------------------------
start_date = "2018-01-01"
end_date = "2025-12-01"

data = yf.download(tickers, start=start_date, end=end_date, auto_adjust=True)["Close"]
data = data.ffill().bfill()  # Fill missing values

# ------------------------------------------------------------
# ALIGN TICKERS (VERY IMPORTANT)
# ------------------------------------------------------------
# Only keep tickers that downloaded successfully
final_tickers = list(data.columns)

# Pull company names in the correct order
company_names = [get_company_name(t) for t in final_tickers]

print("\nTicker → Company mapping:")
for t, n in zip(final_tickers, company_names):
    print(f"{t} = {n}")

# Latest price aligned with final tickers
latest_prices = data.iloc[-1]

# ------------------------------------------------------------
# RETURNS + COVARIANCE
# ------------------------------------------------------------
returns = data.pct_change().dropna()
trading_days = 252

mean_returns = returns.mean() * trading_days
cov_matrix = returns.cov() * trading_days

# ------------------------------------------------------------
# PORTFOLIO METRICS
# ------------------------------------------------------------
def portfolio_performance(weights, mean_returns, cov_matrix, risk_free_rate=0.04):
    weights = np.array(weights)
    ret = np.dot(weights, mean_returns)
    vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe = (ret - risk_free_rate) / vol if vol != 0 else 0
    return ret, vol, sharpe

# ------------------------------------------------------------
# RANDOM PORTFOLIOS
# ------------------------------------------------------------
def generate_random_portfolios(num_portfolios, mean_returns, cov_matrix, risk_free_rate=0.04, seed=42):
    np.random.seed(seed)
    n_assets = len(mean_returns)

    results = {"returns": [], "volatility": [], "sharpe": [], "weights": []}

    for _ in range(num_portfolios):
        weights = np.random.random(n_assets)
        weights /= weights.sum()

        ret, vol, sharpe = portfolio_performance(
            weights, mean_returns.values, cov_matrix.values, risk_free_rate
        )

        results["returns"].append(ret)
        results["volatility"].append(vol)
        results["sharpe"].append(sharpe)
        results["weights"].append(weights)

    return results

num_portfolios = 5000
rp = generate_random_portfolios(num_portfolios, mean_returns, cov_matrix)

returns_array = np.array(rp["returns"])
vol_array = np.array(rp["volatility"])
sharpe_array = np.array(rp["sharpe"])
weights_array = np.array(rp["weights"])

# ------------------------------------------------------------
# OPTIMAL PORTFOLIOS
# ------------------------------------------------------------
max_sharpe_idx = sharpe_array.argmax()
min_vol_idx = vol_array.argmin()

max_sharpe_w = weights_array[max_sharpe_idx]
min_vol_w    = weights_array[min_vol_idx]

max_sharpe_ret, max_sharpe_vol, max_sharpe_ratio = portfolio_performance(
    max_sharpe_w, mean_returns, cov_matrix
)

min_vol_ret, min_vol_vol, min_vol_ratio = portfolio_performance(
    min_vol_w, mean_returns, cov_matrix
)

# ------------------------------------------------------------
# EQUAL WEIGHT PORTFOLIO
# ------------------------------------------------------------
equal_weights = np.ones(len(final_tickers)) / len(final_tickers)
ew_return, ew_vol, ew_sharpe = portfolio_performance(equal_weights, mean_returns, cov_matrix)

# ------------------------------------------------------------
# SHARES + DOLLAR ALLOCATIONS
# ------------------------------------------------------------
alloc_max_sharpe = max_sharpe_w * amount_of_money
alloc_min_vol = min_vol_w * amount_of_money
alloc_equal = equal_weights * amount_of_money

shares_max = alloc_max_sharpe / latest_prices
shares_min = alloc_min_vol / latest_prices
shares_eq  = alloc_equal / latest_prices

# ------------------------------------------------------------
# CREATE OUTPUT TABLE
# ------------------------------------------------------------
allocations_df = pd.DataFrame({
    "Ticker": final_tickers,
    "Company": company_names,
    "LatestPrice": latest_prices.values,
    "MaxSharpeWeight": max_sharpe_w,
    "MaxSharpe$": alloc_max_sharpe,
    "MaxSharpeShares": shares_max,
    "MinVolWeight": min_vol_w,
    "MinVol$": alloc_min_vol,
    "MinVolShares": shares_min,
    "EqualWeight": equal_weights,
    "EqualWeight$": alloc_equal,
    "EqualWeightShares": shares_eq
})

# TOTAL ROW
total_row = pd.DataFrame({
    "Ticker": ["TOTAL"],
    "Company": [""],
    "LatestPrice": [""],
    "MaxSharpeWeight": [max_sharpe_w.sum()],
    "MaxSharpe$": [alloc_max_sharpe.sum()],
    "MaxSharpeShares": [shares_max.sum()],
    "MinVolWeight": [min_vol_w.sum()],
    "MinVol$": [alloc_min_vol.sum()],
    "MinVolShares": [shares_min.sum()],
    "EqualWeight": [equal_weights.sum()],
    "EqualWeight$": [alloc_equal.sum()],
    "EqualWeightShares": [shares_eq.sum()]
})

allocations_df = pd.concat([allocations_df, total_row], ignore_index=True)

# ------------------------------------------------------------
# EXPORT CSV
# ------------------------------------------------------------
allocations_df.to_csv("Portfolio_Optimisation_Results.csv", index=False)
print("\nCSV saved: Portfolio_Optimisation_Results.csv")

# ------------------------------------------------------------
# PLOT + SAVE FRONTIER
# ------------------------------------------------------------
plt.figure(figsize=(10, 7))
plt.scatter(vol_array, returns_array, c=sharpe_array, cmap="viridis", s=10, alpha=0.6)
plt.colorbar(label="Sharpe Ratio")

plt.scatter(max_sharpe_vol, max_sharpe_ret, s=250, marker="*", color="red", label="Max Sharpe")
plt.scatter(min_vol_vol, min_vol_ret, s=250, marker="*", color="blue", label="Min Volatility")

plt.xlabel("Volatility")
plt.ylabel("Expected Return")
plt.title("Efficient Frontier")
plt.legend()
plt.grid(True)
plt.savefig("efficient_frontier.png", dpi=300, bbox_inches="tight")
plt.close()

print("Saved: efficient_frontier.png")

# ------------------------------------------------------------
# PIE CHARTS
# ------------------------------------------------------------
plt.figure(figsize=(8, 8))
plt.pie(max_sharpe_w, labels=final_tickers, autopct="%1.1f%%")
plt.title("Max Sharpe Portfolio")
plt.savefig("max_sharpe_allocation.png", dpi=300, bbox_inches="tight")
plt.close()

plt.figure(figsize=(8, 8))
plt.pie(min_vol_w, labels=final_tickers, autopct="%1.1f%%")
plt.title("Minimum Volatility Portfolio")
plt.savefig("min_vol_allocation.png", dpi=300, bbox_inches="tight")
plt.close()

plt.figure(figsize=(8, 8))
plt.pie(equal_weights, labels=final_tickers, autopct="%1.1f%%")
plt.title("Equal Weight Portfolio")
plt.savefig("equal_weight_allocation.png", dpi=300, bbox_inches="tight")
plt.close()

print("Saved all pie charts.")


What is the total investment amount: $10000


[*********************100%***********************]  8 of 8 completed



Ticker → Company mapping:
AUZ.AX = Australian Mines Limited
BHP.AX = BHP Group Limited
CBA.AX = Commonwealth Bank of Australia
COL.AX = Coles Group Limited
EVN.AX = Evolution Mining Limited
TLS.AX = Telstra Group Limited
WES.AX = Wesfarmers Limited
WOW.AX = Woolworths Group Limited

CSV saved: Portfolio_Optimisation_Results.csv
Saved: efficient_frontier.png
Saved all pie charts.
